In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://53pfvihy4mv-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
from flask import Flask, request, jsonify, render_template
import joblib

app = Flask(__name__, template_folder='/content/drive/MyDrive/predict_dota/draft/lanjutkan/80-20/templates')

# Muat model
model = joblib.load('/content/drive/MyDrive/predict_dota/draft/lanjutkan/80-20/model/xgboost_model.pkl')

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    data = request.form.to_dict()
    data = list(data.values())
    data = list(map(int, data))
    prediction = model.predict_proba([data])[:, 1]
    prediction_value = prediction[0]

    if prediction_value >= 0.5:
        result = 'Team Radiant Menang'
    else:
        result = 'Team Dire Menang'

    return jsonify({
        'prediction': result,
        'percentage': f'{prediction_value * 100:.2f}%'
    })

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [2]:
!pip install flask-ngrok joblib

In [3]:
from flask import Flask, request, jsonify, render_template_string
import joblib
import numpy as np
from flask_ngrok import run_with_ngrok

# Muat model
model = joblib.load('/content/drive/MyDrive/predict_dota/draft/lanjutkan/80-20/model/xgboost_model.pkl')

app = Flask(__name__)
run_with_ngrok(app)  # Ini untuk menjalankan ngrok saat aplikasi Flask berjalan

@app.route('/')
def home():
    return render_template_string('''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Prediksi Tim Pemenang</title>
    </head>
    <body>
        <h1>Prediksi Tim Pemenang</h1>
        <form action="/predict" method="post">
            <label>Masukkan Data (pisahkan dengan koma):</label>
            <input type="text" name="data" placeholder="42, 39, 5, 26, 25, 72, 1, 111, 27, 29, ...">
            <button type="submit">Prediksi</button>
        </form>
        <div id="result"></div>
        <script>
            document.querySelector('form').onsubmit = async (e) => {
                e.preventDefault();
                const formData = new FormData(e.target);
                const response = await fetch('/predict', {
                    method: 'POST',
                    body: formData,
                });
                const result = await response.json();
                document.getElementById('result').innerHTML = `Prediksi: ${result.prediction} dengan Persentase: ${result.percentage}`;
            };
        </script>
    </body>
    </html>
    ''')

@app.route('/predict', methods=['POST'])
def predict():
    data = request.form['data']
    data = data.split(',')
    data = list(map(int, data))
    prediction = model.predict_proba([data])[:, 1]
    prediction_value = prediction[0]

    if prediction_value >= 0.5:
        result = 'Team Radiant Menang'
    else:
        result = 'Team Dire Menang'

    return jsonify({
        'prediction': result,
        'percentage': f'{prediction_value * 100:.2f}%'
    })

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-10:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll